# **Outil de récupération de la liste des informations disponibles dans Wikidata pour un terme de recherche**

Wikidata est une base de connaissances hébergée par la Wikimedia Foundation. Elle permet de centraliser les données utilisées par différents projets Wikimedia. Cet outil est donc utile pour récupérer le maximum d'informations à partir d'un mot clé en questionnant différentes bases de données.

*Entrée* : un terme de recherche comme une personne, un lieu, un monument, une oeuvre, etc.

*Document de sortie* : un fichier texte contenant les informations disponibles dans Wikidata.

In [ ]:
#@markdown # Connexion du notebook à son compte Google Drive et signalement du dossier de travail :

#@markdown - Lancer la cellule
#@markdown - Cliquer sur « Exécuter malgré tout » lors de l’apparition du message d’avertissement indiquant que le notebook n’a pas été créé par Google
#@markdown - Cliquer sur « Se connecter à Google Drive » lors de l’apparition du second message d’avertissement pour donner l’autorisation au notebook d’accéder à vos fichiers Google Drive
#@markdown - Choisir son compte Gmail puis cliquer sur « Autoriser »
''' 
Google Colab notebook.
Python == 3.7.11

BaOIA - La Contemporaine - Université de Nanterre
'''

## Installation des bibliothèques et connexion au compte Google Drive

from google.colab import drive
import re
import glob
import os
import itertools
import json
import urllib.parse, urllib.request, json
import collections
from collections import Counter
!pip install wptools==0.4.17
import wptools

drive.mount('/content/drive/')
%cd /content/drive/My Drive/

#@markdown ###Indiquer le chemin vers le dossier de travail sur le Google Drive (si le dossier n'existe pas, il sera créé lors du lancement de la cellule) :
chemin_vers_le_dossier_de_travail = '/content/drive/My Drive/wikidata/'#@param {type:"string"}

if not os.path.exists(chemin_vers_le_dossier_de_travail):
      os.makedirs(chemin_vers_le_dossier_de_travail)
os.chdir(chemin_vers_le_dossier_de_travail)

In [ ]:
#@markdown # Récupération des informations Wikidata suivant le(s) terme(s) recherché(s) :

#@markdown ####Indiquer le mot clé à rechercher :
mot_cle = "Jean Arp" #@param {type:"string"}
#@markdown ####Changer la langue si nécessaire (par défaut : français) :
langue = "fr" #@param {type:"string"}
#@markdown ####Si la détection du mot clé n'est pas assez précise, descendre le seuil à 0.8. Dans le cas contraire, monter le seuil à 1.0 :
seuil = "1.0" #@param {type:"string"}

def CallWikifier(text, lang=langue, threshold=float(seuil)): 
  # Thresold = taux de sureté. 0.8 = sûr que le résultat est exact; mais peu de résultat.
  # 1 = tous les résultats mais certains faux résultats
    # Url de demande:
    data = urllib.parse.urlencode([
        ("text", text), ("lang", lang),
        ("userKey", "nqvsutgqswvfmrcvyxjtopvpiukjtp"),
        ("pageRankSqThreshold", "%g" % threshold), ("applyPageRankSqThreshold", "true"),
        ("nTopDfValuesToIgnore", "200"), ("nWordsToIgnoreFromList", "200"),
        ("wikiDataClasses", "true"), ("wikiDataClassIds", "false"),
        ("support", "true"), ("ranges", "false"), ("minLinkFrequency", "4"),
        ("includeCosines", "false"), ("maxMentionEntropy", "3")
        ])
    url = "http://www.wikifier.org/annotate-article"
    # Appel de Wikifier 
    req = urllib.request.Request(url, data=data.encode("utf8"), method="POST")
    with urllib.request.urlopen(req, timeout = 500) as f:
        global response
        response = f.read()
        response = json.loads(response.decode("utf8"))
        print(response)
    # Sortie des annotations:
    #@markdown ####Indiquer le nom du fichier créé  avec les informations récoltées grâce à Wikidata :
    infos_wikidata = '/content/drive/My Drive/wikidata/infos_wikidata.txt'#@param {type:"string"}
    with open(infos_wikidata, "w") as outfile:
        for annotation in response["annotations"]:
          itemid = annotation["wikiDataItemId"] 
          page = wptools.page(wikibase=itemid, lang="fr")
          page.get_wikidata()
          page.get_parse()
          infobox = page.data['infobox']
          nom = infobox.get("nom")
          tout = page.data['wikidata']
        json.dump(tout, outfile, indent=4, ensure_ascii=False)

CallWikifier(text=mot_cle)